In [1]:
# Get the list of all the columns that needs to be filled
# Get the list of movie names
# -- Check if the movie has link
# -- If link then get all contents. including infobox, all cast, detect genre, summary, plot etc.
# -- Dump all info on Dataframe and save file. Combine with information on the main page.

In [2]:
import pandas as pd
import ast
import wiki_table_extract as wt
import numpy as np
import os
import re
import json
import wikipedia
from bs4 import BeautifulSoup
import requests

In [3]:
def get_config(dirname, filename):
    """
    Read the config file and create the config obect
    """
    try:
        with open(dirname + filename) as jsonfile:
            config=json.load(jsonfile)
    except Exception as err:
        raise(err)
    return config

config_dir = 'configuration/'
config_file = 'config.json'

config = get_config(config_dir,config_file)

In [4]:
# function to get unordered list from Cast
def get_section_ol_list(url,sectiontitle,titleWiki,fileName,titleYear,entityID):

    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, 'lxml')
    # Added by Arun - Wikitable
    table_classes = {"class": ["mw-headline"],"id":[sectiontitle]}
    wikitables = soup.find("span",table_classes)

    ol_found = 0
    cast_list = []
    c_row_list = []
    
    # if Class of next element is H2 then quit,else search for OL tag,if OL tag is completed then quit
    if wikitables is not None:
        while wikitables.next_element: 
            wikitables = wikitables.next_element
            castName = ""
            castLink = ""
            castRole = ""
            cast_sublist = []

            if wikitables.name == "h2":     
                break
            elif wikitables.name == "ul":
                ol_found = 1
            elif ol_found == 1 and wikitables.name == "li":
                for index,item in enumerate(wikitables.contents):
                    if item.name == "a" and index ==0:
                        castName = item.string
                        cast_sublist.append(castName)
                        castLink = item.get("href")
                        cast_sublist.append(castLink)
                        #print (wt.getLinkandValue(item))
                    elif index == 0:
                        castName = item.string
                        cast_sublist.append(castName)
                        castLink = ""
                        cast_sublist.append(castLink)     
                    elif index == 1:
                        castRole = re.sub(r'([\(\[]).*?([\)\]])', '', item.string)
                        cast_sublist.append(castRole)                      
                cast_list.append(cast_sublist)
            pass

        # Put the castlist as a dictionary
        c_row_list = []

        for cindex,castItem in enumerate(cast_list):

            castdict = {}
            castdict['EntityID'] = entityID
            castdict['WikiTitleLink'] = titleWiki                            
            castdict['Title'] = fileName
            castdict['Year'] = titleYear                                        
            castdict['Role'] = "AllCast"
            castdict['Person'] = re.sub(r'([\(\[]).*?([\)\]])', '', castItem[0])
            if len(castItem) > 2:
                castdict['RoleAlias'] = castItem[2]
            else:
                castdict['RoleAlias'] = ""
            if len(castItem) > 1:
                castdict['PersonWikiLink'] = castItem[1]
            else:
                castdict['PersonWikiLink'] = ""
            # Rowlist to be appended
            c_row_list.append(castdict)
    
    return c_row_list

In [7]:
# Get movies based on indexes. index Range is between 1 to N ( N is the Thousanth digit )

def extract_missing_info(lang,indexRange):

    # Get the file to be used.
    movieDFfile = config['dataframe_dir'].replace('{LANG}',lang) + lang + '_movie_combined_yearly.csv'
    infoboxDir = config['infobox_dir'].replace('{LANG}',lang)

    movieListDF = pd.read_csv(movieDFfile)

    allMovieEntityColumns = ['EntityID','WikiTitleLink','Language','Title','Year','ReleaseDate','Production','Budget','BoxOffice','Runtime']
    # Examples of Person [ Producer,Cast,Director, Music Director, Writters, Screenplay etc ]
    allMoviePersonColumns = ['EntityID','WikiTitleLink','Title','Year','Person','PersonWikiLink','Role','RoleAlias']
    # Examples of Key / Value pairs [ Plot, Summary, End Links ]
    allMovieTextColumns = ['EntityID','WikiTitleLink','Title','Year','Summary','OriginalPlot']

    personList = ['Directed by','Produced by','Written by','Story by','Screenplay by','Starring','Music by','Cinematography','Edited by']
    entityList = ['Release date','Running time','Productioncompany','Budget','BoxOffice']
    entityMaplist = {'Release date':'ReleaseDate','Running time':'Runtime','Productioncompany':'Production','Budget':'Budget','Box office':'BoxOffice'}

    # Data frame for different types of data
    dfMoviePerson = pd.DataFrame(columns=allMoviePersonColumns)
    dfEntityData = pd.DataFrame(columns=allMovieEntityColumns)
    dfTextData = pd.DataFrame(columns=allMovieTextColumns)

    # Iterrate over all the movies in the langauge and collect data from wiki links
    for index,row in movieListDF.iterrows():
        try:
            # Get the films in the 1000's
            if index >= ( ( indexRange - 1 ) * 1000 ) and index < ( indexRange * 1000 ):

                # collect basic info about the movie from the DF

                titleWikiDict = ast.literal_eval(row['Title'])
                titleWikiLink = (next (iter (titleWikiDict.values())))
                titleWiki = (next (iter (titleWikiDict.keys())))
                titleYear = row['TableFilename'].split('/')[3].split('_')[1]            
                fileName = str(titleYear) + '_' + list(titleWikiDict.keys())[0]
                entityID = row['EntityID']

                print (index,infoboxDir,fileName,entityID)

                # Save the infobox and get summary and plot from the Wiki

                if titleWikiLink.startswith('/wiki/'):
                    wikiURL = 'https://en.wikipedia.org' + titleWikiLink
                    wt.scrape(url=wikiURL,output_folder=infoboxDir,filetemplate=fileName,tabletype="infobox")

                    # Get Summary, Cast and Plot from the Wiki page
                    wikipage=wikipedia.WikipediaPage(titleWikiLink.replace('/wiki/',''))
                    wikiPlot = wikipage.section('Plot')
                    wikiSummary = wikipage.summary

                    text_list = []
                    textdict = {}
                    textdict['EntityID']=entityID
                    textdict['WikiTitleLink']=titleWikiLink
                    textdict['Title']=titleWiki
                    textdict['Year']=titleYear
                    textdict['OriginalPlot'] = wikiPlot
                    textdict['Summary'] = wikiSummary

                    # Rowlist to be appended
                    text_list.append(textdict)
                    dfTextData = dfTextData.append(text_list)                     

                    # Get All Cast from the Wiki page
                    c_row_list = get_section_ol_list(wikiURL,"Cast",titleWiki,fileName,titleYear,entityID)

                    if len(c_row_list) > 0:
                        dfMoviePerson = dfMoviePerson.append(c_row_list) 


                # Extract information from the Wiki page that was saved

                e_row_list = []
                entitydict = {}
                entitydict['EntityID']=entityID
                entitydict['WikiTitleLink']=titleWikiLink
                entitydict['Language']="Tamil"
                entitydict['Title']=titleWiki
                entitydict['Year']=titleYear
                entitydict['ReleaseDate']=""
                entitydict['Production']=""
                entitydict['Budget']=""
                entitydict['BoxOffice']="" 
                entitydict['Runtime']=""

                for directory, subdirectories, files in os.walk(infoboxDir):
                    for file in files:
                        if fileName in file:

                            tableFile = os.path.join(infoboxDir, file)
                            tableDF = pd.read_csv(tableFile)

                            # For each item in the infobox
                            for t_index,t_row in tableDF.iterrows():

                                # Try catch to ignore issues when {} and eval function fails. 
                                try:
                                    rowHeader = ast.literal_eval(t_row[1])

                                    # Boolean to test if the Dict is empty or not.
                                    if bool(rowHeader) is True:
                                        rowHeaderKey = (next (iter (rowHeader.keys())))
                                        rowValue = ast.literal_eval(t_row[2])

                                        # Dealing with multiple Keys. Eg. Muti starrer, multi director , writters
                                        for rowValueKey, rowValueLink in rowValue.items():

                                            if rowHeaderKey in personList:

                                                p_row_list = []
                                                persondict = {}
                                                persondict['EntityID']=entityID
                                                persondict['WikiTitleLink'] = titleWiki                            
                                                persondict['Title'] = fileName
                                                persondict['Year'] = titleYear                                        
                                                persondict['Role'] = rowHeaderKey
                                                persondict['RoleAlias'] = ""
                                                persondict['Person'] = re.sub(r'([\(\[]).*?([\)\]])', '', rowValueKey)
                                                persondict['PersonWikiLink'] = rowValueLink

                                                # Rowlist to be appended
                                                p_row_list.append(persondict)
                                                dfMoviePerson = dfMoviePerson.append(p_row_list) 

                                            elif rowHeaderKey in entityList:
                                                colName=entityMaplist[rowHeaderKey]
                                                # Removes () and [] from the text. Was in Date and links like [1]
                                                entitydict[colName]=re.sub(r'([\(\[]).*?([\)\]])', '', rowValueKey)
                                            else:
                                                # Do nothing ( there are other parameters that are not captured for now)
                                                pass

                                except:
                                    print (entityID,titleWikiDict, " Unable to get InfoBox details")
                                    pass

                e_row_list.append(entitydict)
                dfEntityData = dfEntityData.append(e_row_list) 

        except:
            print (entityID,titleWikiDict," Failed")
            pass


    # Save the dataframe
    
    dfMoviePerson.to_csv(config['dataframe_dir'].replace('{LANG}',lang) + lang + "_movie_persons" + str(indexRange) + ".csv")
    dfEntityData.to_csv(config['dataframe_dir'].replace('{LANG}',lang) + lang + "_movie_entites" + str(indexRange) + ".csv")
    dfTextData.to_csv(config['dataframe_dir'].replace('{LANG}',lang) + lang + "_movie_text_contents" + str(indexRange) + ".csv")

    return True

2001 Downloads/Tamil/Infobox/ 1981_Amara Kaaviyam d4611d6e-8847-493d-b429-b917c725a71b
2002 Downloads/Tamil/Infobox/ 1981_Anbulla Atthan b92d998b-7d60-4c06-9e91-9519847ae45a
2003 Downloads/Tamil/Infobox/ 1981_Andhi Mayakkam 75e4523d-44e2-41ce-9a7a-e9512041bc13
2004 Downloads/Tamil/Infobox/ 1981_Andru Muthal Indru Varai c155a6ae-f52a-4ee6-a50c-98f9a53aca6f
2005 Downloads/Tamil/Infobox/ 1981_Anicha Malar 2e33f360-f4c1-459b-8db6-bcb59f6d19b3
2006 Downloads/Tamil/Infobox/ 1981_Antha Ezhu Naatkal 576f4beb-43ff-43a0-9424-9e0433e701ef
2007 Downloads/Tamil/Infobox/ 1981_Anjatha Nenjangal 231780e1-1f75-42e1-8c18-2ae084d9d46b
2008 Downloads/Tamil/Infobox/ 1981_Aradhanai ee685358-4144-43ea-aa72-a88d6c88fdd5
2009 Downloads/Tamil/Infobox/ 1981_Arthangal Ayiram d5d8682e-1ee7-4874-b20b-8dd55a636493
2010 Downloads/Tamil/Infobox/ 1981_Arumbugal 3332592c-51a0-471f-aabc-d699007902bf
2011 Downloads/Tamil/Infobox/ 1981_Avalum Thaayaanal c5f0d7c4-9cc8-409d-aba1-73fa09bfe569
2012 Downloads/Tamil/Infobox/ 198

2095 Downloads/Tamil/Infobox/ 1981_Vaa Intha Pakkam 42031959-0c23-4da0-ac48-384bb1888eab
2096 Downloads/Tamil/Infobox/ 1981_Vaadagai Veedu 431c758d-2a84-4f25-b557-90b7b9e1c118
2097 Downloads/Tamil/Infobox/ 1981_Vasanthakalam f3359492-7d14-40bc-8c83-37015fc474d5
2098 Downloads/Tamil/Infobox/ 1981_Velichathukku Vaanga c19a531f-50ec-47ba-85f7-04096cf5f891
2099 Downloads/Tamil/Infobox/ 1981_Vidiyum Varai Kaathiru 6bc42251-49af-4bed-9d7c-cd0396e9d93d
2100 Downloads/Tamil/Infobox/ 1982_Adhisayappiravigal 52d45b2c-1fa6-470f-a81b-78a629dd7fac
2101 Downloads/Tamil/Infobox/ 1982_Agaya Gangai e3aeffcc-d881-416d-9c1b-b0f707b59411
2102 Downloads/Tamil/Infobox/ 1982_Agni Sakshi 831681f1-4bec-4f55-a15b-9184077502bf
2103 Downloads/Tamil/Infobox/ 1982_Amma 11aa2e9f-b4c1-4af4-b4ca-710a1b976ea1
2104 Downloads/Tamil/Infobox/ 1982_Anandha Ragam 33d33917-80e7-426e-975a-435248a24e78
2105 Downloads/Tamil/Infobox/ 1982_Antha Rathirikku Satchi Illai 15cfe9a7-5ac6-4189-acb0-a6e602130b59
2106 Downloads/Tamil/Info

2188 Downloads/Tamil/Infobox/ 1982_Puthukavithai 425c6f9b-a268-4d95-817f-bb57ed3a922e
2189 Downloads/Tamil/Infobox/ 1982_Raagam Thedum Pallavi d7ecac78-d266-4097-adb2-28bb61da332b
2190 Downloads/Tamil/Infobox/ 1982_Raga Bandhangal a40fe51b-d7ea-463d-8735-7c8dd3d19b4f
2191 Downloads/Tamil/Infobox/ 1982_Ranga 715d7e34-1d81-43db-b434-1ecdaf7365e3
2192 Downloads/Tamil/Infobox/ 1982_Rani Theni 7fb02bfd-c4a2-442d-a3ca-5c1a8c88b411
2193 Downloads/Tamil/Infobox/ 1982_Sakalakala Vallavan b207b966-50e6-4736-a721-2dcf719aee34
2194 Downloads/Tamil/Infobox/ 1982_Sangili 29df4f26-90ac-4d31-a172-b883ff06c9e9
2195 Downloads/Tamil/Infobox/ 1982_Sattam Sirikkiradhu cb0c221a-7f0d-4358-8c7f-9a1f8f97a191
2196 Downloads/Tamil/Infobox/ 1982_Simla Special 240c95fa-52b3-4a6f-8bba-a964b8dfee60
2197 Downloads/Tamil/Infobox/ 1982_Sivantha Kangal cca0828b-a552-4d6a-8f59-b1a31c073542
2198 Downloads/Tamil/Infobox/ 1982_Sparisam 822966e4-184b-463e-9bb6-680d13497160
2199 Downloads/Tamil/Infobox/ 1982_Thaai Mookaambika

2281 Downloads/Tamil/Infobox/ 1983_Salangai Oli 8c835119-0e35-45d3-a9d1-e63abeea267f
2282 Downloads/Tamil/Infobox/ 1983_Samsaram Enbadhu Veenai 93f5ee35-d175-4f29-9af3-e08d09bfcece
2283 Downloads/Tamil/Infobox/ 1983_Sandhippu 738e7f55-ade1-49e5-908a-df38f973152b
2284 Downloads/Tamil/Infobox/ 1983_Saranalayam 2dec6b71-90a2-4bda-80bf-b005f4074b2d
2285 Downloads/Tamil/Infobox/ 1983_Saattai Illatha Pambaram 803e2a85-6382-42db-9693-9de2a642272f
2286 Downloads/Tamil/Infobox/ 1983_Sattathukku Oru Saval c096403a-de23-4178-aac8-f63bbc399b40
2287 Downloads/Tamil/Infobox/ 1983_Sattam ec2521d3-5a52-48e7-94c1-54840ea5c1e5
2288 Downloads/Tamil/Infobox/ 1983_Seerum Singangal 0211a71a-4ac4-45ec-87e4-5227cf6188ab
2289 Downloads/Tamil/Infobox/ 1983_Shasti Viratham 569b16bd-96c3-476b-baec-006b78e45b05
2290 Downloads/Tamil/Infobox/ 1983_Sivappu Sooriyan 1a000c6a-175c-4f6a-87e8-e8338df679ee
2291 Downloads/Tamil/Infobox/ 1983_Silk Silk Silk e567f0e7-c783-460e-867b-9c68925515be
2292 Downloads/Tamil/Infobox/ 

2374 Downloads/Tamil/Infobox/ 1984_Neram Nalla Neram e8537888-20b7-41d2-81a4-386876ae8249
2375 Downloads/Tamil/Infobox/ 1984_Nerupukkul Eeram 462614b9-48a5-42bd-80ac-25f3e7801d11
2376 Downloads/Tamil/Infobox/ 1984_Nichayam 8b0b3cce-73c2-4d6a-a6f7-42261688a80b
2377 Downloads/Tamil/Infobox/ 1984_Nilavu Suduvathillai 229d1b40-b187-47c6-a301-87b55a5650d7
2378 Downloads/Tamil/Infobox/ 1984_Ninaivugal e1d5e51c-ff9a-4546-b800-857ef891968f
2379 Downloads/Tamil/Infobox/ 1984_Nirabarathi b4d6224f-8e54-4348-a500-ef0fe7b90e42
2380 Downloads/Tamil/Infobox/ 1984_Nooravathu Naal d8ffa7e3-c5c9-4e24-86d0-228a3dea171c
2381 Downloads/Tamil/Infobox/ 1984_Nyayam e9bacd03-d60b-493a-98ef-cacbc35b066c
2382 Downloads/Tamil/Infobox/ 1984_Nyayam Ketkiren 90737449-e891-4c3d-a5e2-a5a5156c8e75
2383 Downloads/Tamil/Infobox/ 1984_Oh Maane Maane 111ad66c-81fb-4bea-b43e-bf847aaf1c4f
2384 Downloads/Tamil/Infobox/ 1984_Oomai Janangal f85c54c0-cbc1-4a63-ba37-d0dc91f5f85a
2385 Downloads/Tamil/Infobox/ 1984_Oorukku Ubadesam

...

5988 Downloads/Tamil/Infobox/ 2016_54321 4403a309-c275-4a3d-9b96-84f08d30b3f6
5989 Downloads/Tamil/Infobox/ 2016_Andha Kuyil Needhana a36dc41a-6ff9-496b-a2c3-bac544687ad8
5990 Downloads/Tamil/Infobox/ 2016_Enakku Veru Engum Kilaigal Kidayathu 67c19adc-cee3-4fec-b020-80758b72b4fb
5991 Downloads/Tamil/Infobox/ 2016_Meendum Oru Kadhal Kadhai 3431bf45-d26c-4b55-8df4-665872636961
5992 Downloads/Tamil/Infobox/ 2016_Vendru Varuvaan 85f06afb-ca47-43bf-8120-b3c01029afdf
5993 Downloads/Tamil/Infobox/ 2016_Ilamai Oonjal c2f36ad1-9b23-4cbc-9a26-8427803bf179
5994 Downloads/Tamil/Infobox/ 2016_Kidaari 4d391e98-8013-4d34-82a3-3bfd17824071
5995 Downloads/Tamil/Infobox/ 2016_Kuttrame Thandanai a82c5307-3e31-4ccf-8c47-96b19c323303
5996 Downloads/Tamil/Infobox/ 2016_Thagadu 3fa3817e-0fcc-473f-9b4e-79735653f3c2
5997 Downloads/Tamil/Infobox/ 2016_Iru Mugan 9c2d5a05-b11f-4107-ba0a-a0a6e869a7ad
5998 Downloads/Tamil/Infobox/ 2016_Vaaimai 9225f8ca-b029-4aa5-8871-1a5300a1e7a0
5999 Downloads/Tamil/Infobox/ 2016_

6083 Downloads/Tamil/Infobox/ 2017_Oru Mugathirai c0f3b1be-afd7-404b-9cd7-527ab3eb82f5
6084 Downloads/Tamil/Infobox/ 2017_Vaanga Vaanga 92b86cf9-6880-4caf-86f0-3a1f142c1926
6085 Downloads/Tamil/Infobox/ 2017_1 A.M e41ceca3-b8b0-4408-8a6c-3559bbab19f0
6086 Downloads/Tamil/Infobox/ 2017_465 6513c785-e6ea-4c29-a75d-03db4d7055fe
6087 Downloads/Tamil/Infobox/ 2017_Dhayam 61a419de-2fc8-44c8-a0fb-8e3da20a8dc8
6088 Downloads/Tamil/Infobox/ 2017_Enkitta Mothathe 66271dd4-bb9a-4d34-bd82-4f4397336381
6089 Downloads/Tamil/Infobox/ 2017_Kadugu bbb0edbc-ba12-4fe8-a886-11de4116d6b1
6090 Downloads/Tamil/Infobox/ 2017_Paambhu Sattai 5dfa0451-5650-4ab8-b5af-46f4e46b6372
6091 Downloads/Tamil/Infobox/ 2017_Vaigai Express 66da8deb-2fd6-41c8-8611-4ec7d13ffde4
6092 Downloads/Tamil/Infobox/ 2017_Arasakulam c3d83ff0-9e76-442e-ad46-222e0d743858
6093 Downloads/Tamil/Infobox/ 2017_Attu e22448d4-7038-4f2b-b625-27e2682c9aef
6094 Downloads/Tamil/Infobox/ 2017_Dora 1c588c97-4a4b-4816-846b-9998cafcfb92
6095 Downloads/

6180 Downloads/Tamil/Infobox/ 2017_Oru Kanavu Pola 43ef6894-0c4d-40f1-a12a-4374fc5a4a73
6181 Downloads/Tamil/Infobox/ 2017_Puriyaadha Pudhir 66a9e7ae-18eb-4409-b28e-12730ae8b041
6182 Downloads/Tamil/Infobox/ 2017_Aaram Vetrumai 065132c7-27bd-440f-89aa-3db0b4d56fda
6183 Downloads/Tamil/Infobox/ 2017_Gilli Bambaram Goli 17e535bc-b866-4bf3-9bab-f31df7aa0c22
6184 Downloads/Tamil/Infobox/ 2017_Kadhal Kasakuthaiya 7a732b04-8337-47c4-985f-8f0717c3ce72
6185 Downloads/Tamil/Infobox/ 2017_Kathanayagan 40ea5a32-37a0-4299-a719-08c345401873
6186 Downloads/Tamil/Infobox/ 2017_Maaya Mohini 619b7e6f-0c41-4dea-9857-232929b65f02
6187 Downloads/Tamil/Infobox/ 2017_Neruppu Da 57fb98d3-ac86-4412-b566-3812016f7078
6188 Downloads/Tamil/Infobox/ 2017_Thappu Thanda 6af7bddf-4f23-4070-a22c-52548507b0cf
6189 Downloads/Tamil/Infobox/ 2017_Thupparivaalan 2ab3bc44-5f9f-4ecb-8a5f-bd39d869e707
6190 Downloads/Tamil/Infobox/ 2017_Kombay 7cb98b3a-6c2a-42a7-941f-8e9425125d6a
6191 Downloads/Tamil/Infobox/ 2017_Magalir Mat

6274 Downloads/Tamil/Infobox/ 2017_Iravaakaalam f672107a-c0ea-4bab-b674-2b662137c299
6275 Downloads/Tamil/Infobox/ 2017_Iravukku Aayiram Kangal d694aefa-4744-4a3c-9635-f5145dc02e18
6276 Downloads/Tamil/Infobox/ 2017_Iruttu Araiyil Murattu Kuthu 8a2f1cb5-efc8-4579-9828-9acfe0583f1b
6277 Downloads/Tamil/Infobox/ 2017_Irumbu Thirai 710da068-d97f-48a3-a5f5-f87adcf6334c
6278 Downloads/Tamil/Infobox/ 2017_Jeyikkira Kuthira 29da54bb-da94-4f85-aab0-2bf87919d4eb
6279 Downloads/Tamil/Infobox/ 2017_Johnny c81eb359-b2a3-4168-9998-a6f69756ab15
6280 Downloads/Tamil/Infobox/ 2017_Junga a716eb90-52c4-479c-a1ac-4c9442304120
6281 Downloads/Tamil/Infobox/ 2017_Kaala 57ac8355-c3a6-47ec-89cf-964fc21f450f
6282 Downloads/Tamil/Infobox/ 2017_Kaala Koothu a73d2516-7c0f-4b3a-8daf-269e001d9f2c
6283 Downloads/Tamil/Infobox/ 2017_Kaali 6c4d271a-da87-44f2-a0ac-b8b6ed4adb27
6284 Downloads/Tamil/Infobox/ 2017_Kaateri 355f8909-6034-4788-b44f-35899616f792
6285 Downloads/Tamil/Infobox/ 2017_Kaathadi c94be046-fcd1-4dce-b

6370 Downloads/Tamil/Infobox/ 2017_Server Sundaram aebb3c90-2d65-44bd-81f2-1a4ae9f308a9
6371 Downloads/Tamil/Infobox/ 2017_Shaitan Ka Bachcha 0c575fc6-d3f5-44fd-9aaf-a1791476e5d1
6372 Downloads/Tamil/Infobox/ 2017_Shakthi f23157fb-c72c-4c3b-8c71-62a8de116cee
6373 Downloads/Tamil/Infobox/ 2017_Sigai 4a79e445-a8aa-45ea-851b-e697fd857900
6374 Downloads/Tamil/Infobox/ 2017_Sila Samayangalil f17a55a1-5d33-40f1-b002-255e27b168e3
6375 Downloads/Tamil/Infobox/ 2017_Silukkuvarupatti Singam 1b55ebf5-ae0a-46b9-b835-49f4e4645922
6376 Downloads/Tamil/Infobox/ 2017_Simba 24fb0bd9-524d-49b4-8c72-3660e8644202
6377 Downloads/Tamil/Infobox/ 2017_Sivaranjiniyum Innum Sila Pengalum 9404bfec-bf7c-41fe-a4a3-9c7c7e68ae02
6378 Downloads/Tamil/Infobox/ 2017_Sketch 9a42c38a-43f0-420b-b635-826387c669fa
6379 Downloads/Tamil/Infobox/ 2017_Super Deluxe d8590406-f72e-4e94-8348-94b9f5b7eb5c
6380 Downloads/Tamil/Infobox/ 2017_Thaana Serndha Kootam 901ca682-a838-4eb5-826b-77fcfe28719c
6381 Downloads/Tamil/Infobox/ 2017

In [ ]:
import uuid
print (uuid.uuid4())
import random
print (''.join(random.choice('ARU0123456789VIGNESH') for i in range(8)))

In [ ]:
import dateutil.parser
yourdate = dateutil.parser.parse(dfEntityData['ReleaseDate'].iloc[1])
print (yourdate)

In [ ]:
import scipy.sparse as sps
shape = (1000, 2000)
rows, cols = 1000, 2000
sps_acc = sps.coo_matrix((rows, cols)) # empty matrix
for j in range(100): # add 100 sets of 100 1's
    r = np.random.randint(rows, size=100)
    c = np.random.randint(cols, size=100)
    d = np.ones((100,))
    sps_acc = sps_acc + sps.coo_matrix((d, (r, c)), shape=(rows, cols))

sps_acc

In [ ]:
print (sps_acc.sum(axis=1))

In [ ]:
test ='Downloads/Tamil/Yearly/Decade_1930/0.csv'
print (test.split('/')[3].split('_')[1])


In [ ]:
dfTextData

In [ ]:
titleWikiLink='/wiki/Enthiran'
wikipage=wikipedia.WikipediaPage(titleWikiLink.replace('/wiki/',''))
wikiPlot = wikipage.section('Plot')
wikiSummary = wikipage.summary
wikiCast = wikipage.section('Cast')

print (wikiPlot,wikiSummary,wikiCast)

In [ ]:
import csv
import os
import platform
import pandas as pd
import ast
import json
from bs4 import BeautifulSoup
import requests

url = "https://en.wikipedia.org/wiki/Enthiran"

resp = requests.get(url)
soup = BeautifulSoup(resp.content, 'lxml')
# Added by Arun - Wikitable
table_classes = {"class": ["mw-headline"],"id":["Cast"]}
wikitables = soup.find("span",table_classes)

ol_found = 0
cast_list = []
# if Class of next element is H2 then quit,else search for OL tag,if OL tag is completed then quit
while wikitables.next_element:
    wikitables = wikitables.next_element
    
    if wikitables.name == "h2":     
        break
    elif wikitables.name == "ul":
        ol_found = 1
    elif ol_found == 1 and wikitables.name == "li":
        for item in wikitables.contents:
            print (item)
         cast_list.append(wikitables.contents[0],wikitables.contents[1])
    pass



In [ ]:
        if np.isnan(row['Genre']) is False:
            genreWikiDict = ast.literal_eval(row['Genre'])
            genreWikiLink = (next (iter (genreWikiDict.keys()))).lower()

In [ ]:
print (dfMoviePerson.head())
print (dfEntityData.head())
print (dfTextData.head())


In [ ]:
int(1000/1000)